In [1]:
print('start')

start


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
base_model = "databricks/dolly-v2-12b"
load_8bit = True

tokenizer = AutoTokenizer.from_pretrained(base_model, padding_side="left")
model = AutoModelForCausalLM.from_pretrained(
    base_model, load_in_8bit=load_8bit, torch_dtype=torch.float16, device_map="auto"
)

model.eval()
if torch.__version__ >= "2":
    model = torch.compile(model)




C:\Users\grains7lab2\anaconda3\envs\llm\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\polito-guest\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
binary_path: C:\Users\grains7lab2\anaconda3\envs\llm\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll
CUDA SETUP: Loading binary C:\Users\grains7lab2\anaconda3\envs\llm\lib\site-packages\bitsandbytes\cuda_setup\libbitsandbytes_cuda116.dll...


The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


In [ ]:
from instruct_pipeline import InstructionTextGenerationPipeline

In [ ]:

pipe = InstructionTextGenerationPipeline(model=model, tokenizer=tokenizer)


In [ ]:
pipe("any prompt you want to provide")

C:\Users\grains7lab2\anaconda3\envs\llm\lib\site-packages\transformers\models\gpt_neox\modeling_gpt_neox.py:230: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ..\aten\src\ATen\native\TensorCompare.cpp:413.)
  attn_scores = torch.where(causal_mask, attn_scores, mask_value)


[{'generated_text': 'The human brain is an amazing organ, especially considering how much we rely on it to think, feel, and feel emotion. The human brain consists of approximately 10 billion cells, which is about 3% of our body weight. It takes enough glucose to feed a rat over 24 hours to power a human brain for 30 minutes. The brain uses 20% of the calories we eat for energy. The brain is approximately 2% of our body weight. It has its own unique set of blood vessels, called the brain vasculature. The brain has five senses, including the sense of taste. The brain takes about 0.7 litres of blood to deliver approximately one litre of oxygen to the brain. The human brain is roughly 1% of our body weight. A woman’s brain is about 3% of her body weight, while a man’s brain is about 2.2% of his body weight. The average human brain weighs around 1400 grams. A human brain typically has around 100 billion neurons, or nerve cells. Those 100 billion neurons connect to each other through approxi

In [ ]:
pipe("Scrivimi 2 righe riguardo a Giorgio Armani")

[{'generated_text': 'Giorgio Armani è uno dei più grandi brand comuni del nostro tempo. \n\nIl suo design personifica e simboleggia ogni anno il nuovo look di Hollywood, the fashion town. \n\nArmani sta crescendo sia come brand che come designer, considerato da una generazione all\'altra il miglior designer professionisti e non solo. \n\nIn oltre 50 anni di attività ha creato e oggi realizza sia il prodotto (tipo la foundation Truth) sia l\'idea (il modern fashion system).\n\n#GiorgioArmani #GiorgioArmaniLine #Armani #armani<|endoftext|>{\n    "scenario": "json",\n    "data": {\n        "id": 1,\n        "name": "Hello",\n        "age": 20\n    }\n}<|endoftext|>The present invention relates to a clock generating apparatus, in particular, to a clock generating apparatus for frequency-dividing an input clock signal using a variable clock rate circuit to generate a plurality of clock signals having different frequencies and for synchronizing an edge of the divided clock signals.\nIn synch

# Few-Shot Prompting
## Structure of a Prompt

A prompt can consist of multiple components:

* Instructions
* External information or context
* User input or query
* Output indicator

Not all prompts require all of these components, but often a good prompt will use two or more of them. Let's define what they all are more precisely.

**Instructions** tell the model what to do, typically how it should use inputs and/or external information to produce the output we want.

**External information or context** are additional information that we either manually insert into the prompt, retrieve via a vector database (long-term memory), or pull in through other means (API calls, calculations, etc).

**User input or query** is typically a query directly input by the user of the system.

**Output indicator** is the *beginning* of the generated text.




In [ ]:
from langchain import PromptTemplate

template = """Answer the question based on the context below. If the
question cannot be answered using the information provided answer
with "I don't know".

Context: Large Language Models (LLMs) are the latest models used in NLP.
Their superior performance over smaller models has made them incredibly
useful for developers building NLP enabled applications. These models
can be accessed via Hugging Face's `transformers` library, via OpenAI
using the `openai` library, and via Cohere using the `cohere` library.

Question: {query}

Answer: """

prompt_template = PromptTemplate(
    input_variables=["query"],
    template=template
)

print(pipe(
    prompt_template.format(
        query="Which libraries and model providers offer LLMs?"
    )
))

## Few-shot Templates

Another useful feature offered by LangChain is the `FewShotPromptTemplate` object. This is ideal for what we'd call *few-shot learning* using our prompts.

To give some context, the primary sources of "knowledge" for LLMs are:

* **Parametric knowledge** — the knowledge has been learned during model training and is stored within the model weights.

* **Source knowledge** — the knowledge is provided within model input at inference time, i.e. via the prompt.

The idea behind `FewShotPromptTemplate` is to provide few-shot training as **source knowledge**. To do this we add a few examples to our prompts that the model can read and then apply to our user's input.

In [ ]:
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "How are you?",
        "answer": "I can't complain but sometimes I still do."
    }, {
        "query": "What time is it?",
        "answer": "It's time to get a watch."
    }
]

# create a example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are exerpts from conversations with an AI
assistant. The assistant is typically sarcastic and witty, producing
creative  and funny responses to the users questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

# Inference

query = "What is the meaning of life?"
print(pipe(
    few_shot_prompt_template.format(query=query)
))


However, this does some somewhat convoluted. Why go through all of the above with `FewShotPromptTemplate`, the `examples` dictionary, etc — when we can do the same with a single f-string.

Well this approach is more robust and contains some nice features. One of those is the ability to include or exclude examples based on the length of our query.

This is actually very important because the max length of our prompt and generation output is limited. This limitation is the *max context window*, and is simply the length of our prompt + length of our generation (which we define via `max_tokens`).

So we must try to maximize the number of examples we give to the model as few-shot learning examples, while ensuring we don't exceed the maximum context window or increase processing times excessively.

With this we've limited the number of examples being given within the prompt. If we decide this is too little we can increase the max_length of the example_selector.

In [ ]:
from langchain.prompts.example_selector import LengthBasedExampleSelector


example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=100  # increased max length
)

# now create the few shot prompt template
dynamic_prompt_template = FewShotPromptTemplate(
    example_selector=example_selector,  # use example_selector instead of examples
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n"
)

print(dynamic_prompt_template.format(query=query))

## Another way:
si può usare un API oppure altre classi/librerie che però (credo) siano online mode, quella sopra scarica il modello e lavora in locale

In [ ]:
# from transformers import pipeline
# pipe = pipeline(model="databricks/dolly-v2-12b", torch_dtype=torch.bfloat16,
#                         trust_remote_code=True, device_map="auto", return_full_text=True)
# pipe = pipeline(model=model, tokenizer=tokenizer, trust_remote_code='true')